In [1]:
import pickle
from collections import Counter

# File path to the .pkl file
input_file = "states3.pkl"

# Load the states from the binary file
with open(input_file, 'rb') as pklfile:
    states = pickle.load(pklfile)

print(f"Loaded {len(states)} states from {input_file}")


Loaded 4050380 states from states3.pkl


In [2]:
# import sys
# import os

# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

# from blackjack_game.dealer import DealerPolicy

NUM_DECKS = 1

In [3]:
states

[(-1, 0),
 ((11, 11), -1),
 ((11, 11), (11,)),
 ((11, 11), (2,)),
 ((11, 11), (3,)),
 ((11, 11), (4,)),
 ((11, 11), (5,)),
 ((11, 11), (6,)),
 ((11, 11), (7,)),
 ((11, 11), (8,)),
 ((11, 11), (9,)),
 ((11, 11), (10,)),
 ((11, 11), (10,)),
 ((11, 11), (10,)),
 ((11, 11), (10,)),
 ((11, 11), (11, 11)),
 ((11, 11), (11, 2)),
 ((11, 11), (11, 3)),
 ((11, 11), (11, 4)),
 ((11, 11), (11, 5)),
 ((11, 11), (11, 6)),
 ((11, 11), (11, 7)),
 ((11, 11), (11, 8)),
 ((11, 11), (11, 9)),
 ((11, 11), (11, 10)),
 ((11, 11), (11, 10)),
 ((11, 11), (11, 10)),
 ((11, 11), (11, 10)),
 ((11, 11), (2, 2)),
 ((11, 11), (2, 3)),
 ((11, 11), (2, 4)),
 ((11, 11), (2, 5)),
 ((11, 11), (2, 6)),
 ((11, 11), (2, 7)),
 ((11, 11), (2, 8)),
 ((11, 11), (2, 9)),
 ((11, 11), (2, 10)),
 ((11, 11), (2, 10)),
 ((11, 11), (2, 10)),
 ((11, 11), (2, 10)),
 ((11, 11), (3, 3)),
 ((11, 11), (3, 4)),
 ((11, 11), (3, 5)),
 ((11, 11), (3, 6)),
 ((11, 11), (3, 7)),
 ((11, 11), (3, 8)),
 ((11, 11), (3, 9)),
 ((11, 11), (3, 10)),
 ((11

In [3]:
class DealerPolicy:
    def make_decision(self, score):
        if score < 17:
            return "hit"
        else:
            # print("Dealer stands")
            return "stand"

In [59]:
#deck_cards = [2,3,4,5,6,7,8,9,10,10,10,10,11] * NUM_DECKS

#deck_cards = [2,3,9,10,11] * 3 * NUM_DECKS

deck_cards = [2,3,4,5,6,7,8,9,10,10,10,10,11] * 4 * NUM_DECKS


In [60]:
class State:
    def __init__(self, state):
        #self.original_state = state
        self.player_cards = list(state[0])
        self.dealer_cards = list(state[1])
        self.order_cards()
        self.player_value = self.get_player_value()
        self.dealer_value = self.get_dealer_value()

    def order_cards(self):
        self.player_cards = sorted(self.player_cards)
        self.dealer_cards = sorted(self.dealer_cards)

    def get_cards(self):
        return self.player_cards + self.dealer_cards
    
    def get_player_cards(self):
        return self.player_cards
    
    def get_dealer_cards(self):
        return self.dealer_cards
    
    def is_player_bust(self):
        return self == State(((-1,), (0,)))
    
    def is_dealer_bust(self):
        return self.dealer_cards == [-1]
    
    def player_has_blackjack(self):
        return self.player_value == 21 and len(self.player_cards) == 2
    
    def dealer_has_blackjack(self):
        return self.dealer_value == 21 and len(self.dealer_cards) == 2
    
    def is_bust(self):
        return self.is_player_bust() or self.is_dealer_bust()
    
    def valid_draws(self, deck_cards):
        current_deck = Counter(deck_cards)
        current_cards = Counter(self.get_cards())
        remaining_cards = list((current_deck - current_cards).elements())
        return list(set(remaining_cards))

    
    def get_player_value(self):
        num_aces = 0
        value = 0
        for card in self.player_cards:
            if card == 11:
                num_aces += 1
            value += card
        while value > 21 and num_aces >0:
            value -= 10
            num_aces -= 1
        if value > 21:
            self.player_cards = [-1]
            self.dealer_cards = [0]
            self.player_value = -1
            self.dealer_value = 0
        return value
    
    def num_aces(self):
        return self.player_cards.count(11)

    def get_dealer_value(self):
        num_aces = 0
        value = 0
        for card in self.dealer_cards:
            if card == 11:
                num_aces += 1
            value += card
        while value > 21 and num_aces >0:
            value -= 10
            num_aces -= 1
        if value > 21:
            self.dealer_cards = [-1]
            self.dealer_value = -1
        return value
    
    def get_hit_states(self):
        hit_states = []

        if self.is_bust():
            return hit_states

        if self.player_value == 21:
            return hit_states
        
        else:
            new_states = [State((self.player_cards + [card], self.dealer_cards)) for card in self.valid_draws(deck_cards)]
            if all([state.is_player_bust() for state in new_states]):
                return hit_states
            
            if any([state.is_player_bust() for state in new_states]):
                hit_states.append(State(((-1,), (0,))))

            hit_states += [state for state in new_states if state.player_value <= 21 and not state.is_player_bust()]

            # for card in self.valid_draws(deck_cards):
            #     new_state = State((self.player_cards + [card], self.dealer_cards))

            #     if new_state.is_player_bust():
            #         if new_state not in hit_states:
            #             hit_states.append(new_state)

            #     elif new_state.player_value <= 21:
            #         hit_states.append(new_state)
                 

        return hit_states
 
    def get_dealer_states(self, dealer_policy):
        dealer_states = []
        if self.is_bust() or ((self.get_player_value() <= 12) and (self.num_aces() == 0)):
            return dealer_states
        
        if dealer_policy.make_decision(self.dealer_value) == "hit":
            for card in self.valid_draws(deck_cards):
                new_state = State((self.player_cards, self.dealer_cards + [card]))

                if new_state.is_dealer_bust():
                    if new_state not in dealer_states:
                        dealer_states.append(new_state)

                if new_state.dealer_value <= 21:
                    dealer_states.append(new_state)
                # else:
                #     bust_state = State(((-1,), (0,)))
                #     if (bust_state not in dealer_states):
                #         dealer_states.append(bust_state)
        return dealer_states
    
    def get_card_probs(self, new_card):
        current_deck = Counter(deck_cards)
        current_cards = Counter(self.get_cards())
        remaining_cards = list((current_deck - current_cards).elements())
        return round((remaining_cards.count(new_card) / len(remaining_cards)),3)

    def get_state_prob(self, new_state):
        old_cards = Counter(self.get_cards())
        new_cards = Counter(new_state.get_cards())
        new_card = list((new_cards - old_cards).elements())[0]
        return self.get_card_probs(new_card)
       
    def get_probs(self, input_states):
        if input_states == []:
            return []
        
        probs = []
        states_with_probs = []
        bust_idx = -1
        for idx, state in enumerate(input_states):
            if state.is_player_bust() or state.is_dealer_bust():
                bust_idx = idx
            else:
                #print(state.get_cards(), self.get_cards())
                current_state_count = Counter(self.get_cards())
                new_state_count = Counter(state.get_cards())
                new_card = list((new_state_count - current_state_count).elements())[0]
                prob = self.get_card_probs(new_card)
                probs.append(prob)
                
                states_with_probs.append((state, prob))
        #print(f"State: {self}, Probs: {probs}")

        if bust_idx != -1:
            total_prob = 1.0 - sum(probs)
            states_with_probs.append((input_states[bust_idx], total_prob))

        return states_with_probs

    def __eq__(self, other):
        return self.player_cards == other.player_cards and self.dealer_cards == other.dealer_cards
    
    def __repr__(self):
        return f"Player: {self.player_cards}, Dealer: {self.dealer_cards}"
    
    def __hash__(self):
        return hash((tuple(self.player_cards), tuple(self.dealer_cards)))

In [61]:
def get_starting_states(deck_cards):
    starting_states = [State(((-1,),(0,)))]

    unique_cards = list(set(deck_cards))

    for player_card1 in unique_cards:
        for player_card2 in unique_cards:
            bust_state = State(((player_card1, player_card2), (-1,)))
            if bust_state not in starting_states:
                starting_states.append(bust_state)
            
            for dealer_card in range(2,12):
                new_state = State(((player_card1, player_card2), (dealer_card,)))
                if new_state not in starting_states:
                    starting_states.append(new_state)

    return starting_states


starting_states = get_starting_states(deck_cards)

player_dict = {}
dealer_dict = {}
dealer_policy = DealerPolicy()
def build_transition_dict(starting_states, player_turn = True):
    states_not_in_dict = []
    if player_turn:
        if player_dict.keys():
            states_not_in_dict = [state for state in starting_states]
        else:
            states_not_in_dict = starting_states
        if not states_not_in_dict:
            return
        
    else:
        states_not_in_dict = [state for state in starting_states if state not in dealer_dict]
        if not states_not_in_dict:
            return
    
    for state in states_not_in_dict:
        if player_turn:
            hit_states = state.get_hit_states()
            stand_states = state.get_dealer_states(DealerPolicy())
            player_dict[state] = {
                1: state.get_probs(hit_states),
                0: state.get_probs(stand_states)
            }
            build_transition_dict(hit_states)
            build_transition_dict(stand_states, player_turn = False)

        else:
            dealer_states = state.get_dealer_states(DealerPolicy())
            # print(f'Current State: {state}')
            # print(f'Dealer States probs: {state.get_probs(dealer_states)}')
            # print('\n')
            dealer_dict[state] = state.get_probs(dealer_states)

            build_transition_dict(dealer_states, player_turn = False)


In [62]:
# issues:
# 1. does not seem to consider 11 as an ace
# for some reason: when a card is already present in the state, it is not considered in the remaining cards

In [63]:
build_transition_dict(starting_states)

In [14]:
player_dict[State(((2,7,4), (7,)))]

KeyError: Player: [2, 4, 7], Dealer: [7]

In [64]:
player_dict

{Player: [-1], Dealer: [0]: {1: [], 0: []},
 Player: [2, 2], Dealer: [-1]: {1: [], 0: []},
 Player: [2, 2], Dealer: [2]: {1: [(Player: [2, 2, 2], Dealer: [2], 0.02),
   (Player: [2, 2, 3], Dealer: [2], 0.082),
   (Player: [2, 2, 4], Dealer: [2], 0.082),
   (Player: [2, 2, 5], Dealer: [2], 0.082),
   (Player: [2, 2, 6], Dealer: [2], 0.082),
   (Player: [2, 2, 7], Dealer: [2], 0.082),
   (Player: [2, 2, 8], Dealer: [2], 0.082),
   (Player: [2, 2, 9], Dealer: [2], 0.082),
   (Player: [2, 2, 10], Dealer: [2], 0.327),
   (Player: [2, 2, 11], Dealer: [2], 0.082)],
  0: []},
 Player: [2, 2, 2], Dealer: [2]: {1: [(Player: [2, 2, 2, 3], Dealer: [2],
    0.083),
   (Player: [2, 2, 2, 4], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 5], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 6], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 7], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 8], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 9], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 10], Dealer: [2], 0.333),
   (Playe

In [67]:
with open('player_transitions.pkl','wb') as f:
    pickle.dump(player_dict, f)

with open('dealer_transitions.pkl','wb') as f:
    pickle.dump(dealer_dict, f)

In [120]:
len(player_dict)

19676

In [121]:
len(dealer_dict)

2210871

In [65]:
player_dict

{Player: [-1], Dealer: [0]: {1: [], 0: []},
 Player: [2, 2], Dealer: [-1]: {1: [], 0: []},
 Player: [2, 2], Dealer: [2]: {1: [(Player: [2, 2, 2], Dealer: [2], 0.02),
   (Player: [2, 2, 3], Dealer: [2], 0.082),
   (Player: [2, 2, 4], Dealer: [2], 0.082),
   (Player: [2, 2, 5], Dealer: [2], 0.082),
   (Player: [2, 2, 6], Dealer: [2], 0.082),
   (Player: [2, 2, 7], Dealer: [2], 0.082),
   (Player: [2, 2, 8], Dealer: [2], 0.082),
   (Player: [2, 2, 9], Dealer: [2], 0.082),
   (Player: [2, 2, 10], Dealer: [2], 0.327),
   (Player: [2, 2, 11], Dealer: [2], 0.082)],
  0: []},
 Player: [2, 2, 2], Dealer: [2]: {1: [(Player: [2, 2, 2, 3], Dealer: [2],
    0.083),
   (Player: [2, 2, 2, 4], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 5], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 6], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 7], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 8], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 9], Dealer: [2], 0.083),
   (Player: [2, 2, 2, 10], Dealer: [2], 0.333),
   (Playe

In [66]:
dealer_dict

{Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4]: [(Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 4],
   0.075),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 5], 0.1),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 6], 0.1),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 7], 0.1),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 8], 0.1),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 9], 0.1),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 10], 0.4),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 11], 0.025)],
 Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 4]: [(Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 4, 4],
   0.051),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 4, 5], 0.103),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 4, 6], 0.103),
  (Player: [2, 2, 2, 3, 3, 3, 3, 11, 11, 11], Dealer: [2, 4, 4, 7], 0.

In [113]:


# def get_cards(state):
#     cards = []
#     for card in state[0]:
#         cards.append(card)
#     for card in state[1]:
#         cards.append(card)
#     return cards

# def get_player_cards(state):
#     cards = []
#     for card in state[0]:
#         cards.append(card)
#     return cards

# def get_dealer_cards(state):
#     cards = []
#     for card in state[1]:
#         cards.append(card)
#     return cards

# def get_player_value(state):
#     num_aces = 0
#     value = 0
#     for card in state[0]:
#         if card == 11:
#             num_aces += 1
#         value += card
#     while value > 21 and num_aces > 0:
#         value -= 10
#         num_aces -= 1
#     return value

# def get_dealer_value(state):
#     num_aces = 0
#     value = 0
#     for card in state[1]:
#         if card == 11:
#             num_aces += 1
#         value += card
#     while value > 21 and num_aces > 0:
#         value -= 10
#         num_aces -= 1
#     return value

# def get_hit_states(state):
#     hit_states = []
#     player_value = get_player_value(state)
#     if player_value == 21:
#         return hit_states
#     else:
#         for card in range(2,12):
#             new_state = (state[0] + (card,), state[1])
#             new_player_value = get_player_value(new_state)
#             if new_player_value <= 21:
#                 hit_states.append(new_state)
#             else:
#                 if ((-1, 0) not in hit_states):
#                     hit_states.append((-1, 0))
#     return hit_states

# def get_dealer_states(state, dealer_policy):
#     dealer_score = get_dealer_value(state)
#     player_state = state[0]
#     dealer_states = []
#     if dealer_policy.make_decision(dealer_score) == 'hit':
#         for card in range(2,12):
#             new_state = (state[0], state[1] + (card,))
#             new_dealer_score = get_dealer_value(new_state)
#             if new_dealer_score <= 21:
#                 dealer_states.append(new_state)
#             else:
#                 if ((player_state, -1) not in dealer_states):
#                     dealer_states.append((player_state, -1))
#     return dealer_states

# def get_state_prob(state, new_state):
#     old_cards = get_cards(state)
#     new_cards = get_cards(new_state)
#     new_card = [card for card in new_cards if card not in old_cards][0]
#     remaining_cards = [card for card in deck_cards if card not in old_cards]
#     return remaining_cards.count(new_card) / len(remaining_cards)

# def get_probs(input_states):
#     if input_states == []:
#         return []
#     total_probs = 1
#     states_with_probs = []
#     bust_idx = -1
#     for idx, state in enumerate(input_states):
#         if state == (-1, 0):
#             bust_idx = idx
#         else:
#             prob = get_card_probs(state, 11)
#             total_probs -= prob
#             states_with_probs.append((state, prob))

#     if bust_idx != -1:
#         states_with_probs.append((input_states[bust_idx], total_probs))
        

# def get_card_probs(state, new_card):
#     cards = get_cards(state)
#     remaining_cards = [card for card in deck_cards if card not in cards]
#     return remaining_cards.count(new_card) / len(remaining_cards)   


